In [ ]:
import pandas as pd

df = pd.read_csv("../../../data/wsdm_mini.csv")
df.head()

In [ ]:
# 合并文本，然后对文本进行分词处理 

df['title_zh'] = df[['title1_zh', 'title2_zh']].apply(lambda x: ''.join(x), axis=1)  # 合并文本数据列
df_merge = df.drop(df.columns[[0, 1]], axis=1)  # 删除原文本列
df_merge.head()

In [ ]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip('\n') for line in f.readlines()]
    return stopwords
stopwords = load_stopwords('../../../data/stopwords.txt')
stopwords

In [ ]:
from tqdm.notebook import tqdm

# 利用 jieba 进行分词，并且取出停用词

corpus = []
for line in tqdm(df['title_zh']):
    words = []
    seg_list = list(jieba.cut(line))  # 分词
    for word in seg_list:
        if word in stopwords:  # 删除停用词
            continue
        words.append(word)
    corpus.append(words)

In [ ]:
import numpy as np
from gensim.models import Word2Vec

model = Word2Vec(corpus)  # 词嵌入，默认 size=100

# 整个句子所有单词向量的和，得到空间向量上这个句子的向量
def sum_vec(text):
    vec = np.zeros(100).reshape((1, 100))  # 初始化一个和 Word2Vec 嵌入等长度的 0 向量
    for word in text:
        # 得到句子中每个词的词向量并累加在一起
        if word in list(model.wv.index_to_key):
            vec += model.wv.get_vector(word).reshape((1, 100))
        else:
            pass
    return vec

# 将词向量保存为 Ndarray
X = np.concatenate([sum_vec(z) for z in tqdm(corpus)])
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(X, df.label, test_size=0.2)

# 随机森林算法进行分类
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)